In [23]:
import pandas as pd
import matplotlib.pyplot as plt
import pyarrow.parquet as pq
import geojson
import plotly.express as px
import plotly.io as pio
import json

In [24]:
def load_trips(list):
    df = pd.DataFrame()
    for month in list:
        print(month)
        trips = pq.read_table(f'data/yellow_tripdata_2022-{month:02}.parquet')
        trips_df = trips.to_pandas()
        # Append data
        df = pd.concat((df, trips_df), ignore_index=True)# df.concat(trips_df, ignore_index=True)

    return df
list = range(1,13)
df = load_trips(list)
#df.to_csv('data/yellow_tripdata_combined.csv')
#df = pd.read_csv('data/yellow_tripdata_combined.csv')
print(df.shape)

1
2
3
4
5
6
7
8
9
10
11
12
(39656098, 19)


In [25]:
df["pickup_at"] = pd.to_datetime(df["tpep_pickup_datetime"])
df["dropoff_at"] = pd.to_datetime(df["tpep_dropoff_datetime"])

In [26]:
df["duration"] = (df["dropoff_at"] - df["pickup_at"]).dt.total_seconds()

In [27]:
print(df.columns)

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee', 'pickup_at',
       'dropoff_at', 'duration'],
      dtype='object')


In [28]:
df.describe()

,VendorID,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
count,3.965610e+07,3.828780e+07,3.965610e+07,3.828780e+07,3.965610e+07,3.965610e+07,3.965610e+07,3.965610e+07,3.965610e+07,3.965610e+07,3.965610e+07,3.965610e+07,3.965610e+07,3.965610e+07,3.828780e+07,3.828780e+07,3.965610e+07
mean,1.721802e+00,1.401149e+00,5.959399e+00,1.424172e+00,1.648660e+02,1.625752e+02,1.189550e+00,1.036282e+01,1.007532e+00,4.888064e-01,7.234908e+00,5.362827e-01,3.161243e-01,2.167127e+01,2.281437e+00,9.741937e-02,3.209844e+03
std,4.805267e-01,9.628938e-01,5.991907e+02,5.794343e+00,6.531082e+01,7.023146e+01,5.190411e-01,2.232830e+04,1.262564e+00,9.380395e-02,2.232808e+04,2.043351e+00,1.293323e-01,9.637360e+01,7.519435e-01,3.385391e-01,1.159487e+06
min,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,-1.333914e+08,-2.218000e+01,-5.500000e-01,-4.100000e+02,-9.999000e+01,-1.000000e+00,-2.567800e+03,-2.500000e+00,-1.250000e+00,-4.305596e+06
25%,1.000000e+00,1.000000e+00,1.100000e+00,1.000000e+00,1.320000e+02,1.130000e+02,1.000000e+00,7.000000e+00,0.000000e+00,5.000000e-01,9.300000e-01,0.000000e+00,3.000000e-01,1.230000e+01,2.500000e+00,0.000000e+00,4.460000e+02
50%,2.000000e+00,1.000000e+00,1.900000e+00,1.000000e+00,1.620000e+02,1.620000e+02,1.000000e+00,1.050000e+01,5.000000e-01,5.000000e-01,2.160000e+00,0.000000e+00,3.000000e-01,1.596000e+01,2.500000e+00,0.000000e+00,7.360000e+02
75%,2.000000e+00,1.000000e+00,3.560000e+00,1.000000e+00,2.340000e+02,2.340000e+02,1.000000e+00,1.650000e+01,2.500000e+00,5.000000e-01,3.360000e+00,0.000000e+00,3.000000e-01,2.316000e+01,2.500000e+00,0.000000e+00,1.194000e+03
max,6.000000e+00,9.000000e+00,3.896785e+05,9.900000e+01,2.650000e+02,2.650000e+02,5.000000e+00,4.010923e+05,3.350000e+01,2.548000e+01,1.333914e+08,9.118700e+02,1.000000e+00,4.010956e+05,2.750000e+00,1.250000e+00,6.193233e+08


In [29]:
df = df.dropna()
print(df.shape)

(38287795, 22)


In [30]:
df.describe()

,VendorID,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
count,3.828780e+07,3.828780e+07,3.828780e+07,3.828780e+07,3.828780e+07,3.828780e+07,3.828780e+07,3.828780e+07,3.828780e+07,3.828780e+07,3.828780e+07,3.828780e+07,3.828780e+07,3.828780e+07,3.828780e+07,3.828780e+07,3.828780e+07
mean,1.713156e+00,1.401149e+00,3.514399e+00,1.424172e+00,1.649293e+02,1.628563e+02,1.232061e+00,1.462763e+01,1.040985e+00,4.885367e-01,2.713900e+00,5.310589e-01,3.160877e-01,2.142373e+01,2.281437e+00,9.741937e-02,3.281145e+03
std,4.522879e-01,9.628938e-01,5.643246e+01,5.794343e+00,6.494462e+01,7.015708e+01,4.760829e-01,9.743135e+01,1.271069e+00,9.508469e-02,3.288837e+00,2.031171e+00,1.299794e-01,9.801355e+01,7.519435e-01,3.385391e-01,1.180023e+06
min,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,-2.564000e+03,-2.218000e+01,-5.500000e-01,-4.100000e+02,-9.999000e+01,-1.000000e+00,-2.567800e+03,-2.500000e+00,-1.250000e+00,-4.305596e+06
25%,1.000000e+00,1.000000e+00,1.100000e+00,1.000000e+00,1.320000e+02,1.130000e+02,1.000000e+00,7.000000e+00,0.000000e+00,5.000000e-01,7.000000e-01,0.000000e+00,3.000000e-01,1.230000e+01,2.500000e+00,0.000000e+00,4.410000e+02
50%,2.000000e+00,1.000000e+00,1.860000e+00,1.000000e+00,1.620000e+02,1.620000e+02,1.000000e+00,1.000000e+01,5.000000e-01,5.000000e-01,2.150000e+00,0.000000e+00,3.000000e-01,1.595000e+01,2.500000e+00,0.000000e+00,7.280000e+02
75%,2.000000e+00,1.000000e+00,3.490000e+00,1.000000e+00,2.340000e+02,2.340000e+02,1.000000e+00,1.600000e+01,2.500000e+00,5.000000e-01,3.350000e+00,0.000000e+00,3.000000e-01,2.277000e+01,2.500000e+00,0.000000e+00,1.180000e+03
max,2.000000e+00,9.000000e+00,1.843408e+05,9.900000e+01,2.650000e+02,2.650000e+02,5.000000e+00,4.010923e+05,3.350000e+01,2.548000e+01,1.400160e+03,9.118700e+02,1.000000e+00,4.010956e+05,2.750000e+00,1.250000e+00,6.193233e+08


In [31]:
def remove_outliers(df, attribute, min = None, max = None, include_min = True, include_max = True):
    if min is not None:
        if include_min:
            df = df[df[attribute]>=min]
        else:
            df = df[df[attribute]>min]
    if max is not None:
        if include_max:
            df = df[df[attribute]<=max]
        else:
            df = df[df[attribute]<max]
    return df

In [32]:
df = remove_outliers(df, "duration", min = 60, max = 10800)

In [33]:
df = remove_outliers(df, "passenger_count", max = 6)

In [34]:
df = remove_outliers(df, "trip_distance", min = 0, max = 100, include_min = False)

In [35]:
df = remove_outliers(df, "RatecodeID", min = 1, max = 6)

In [36]:
df = remove_outliers(df, "payment_type", min = 1, max = 2)

In [37]:
df = remove_outliers(df, "fare_amount", min = 2.5, max = 250, include_min = False)

In [38]:
df = remove_outliers(df, "extra", min = 0)

In [39]:
df = remove_outliers(df, "mta_tax", min = 0)

In [40]:
df = remove_outliers(df, "tip_amount", min = 0, max = 125)

In [41]:
df = remove_outliers(df, "tolls_amount", min = 0, max = 100)

In [42]:
df = remove_outliers(df, "improvement_surcharge", min = 0)

In [43]:
df = remove_outliers(df, "total_amount", min = 2.5, max = 1000, include_min=False)

In [44]:
df = remove_outliers(df, "congestion_surcharge", min = 0)

In [45]:
df = remove_outliers(df, "airport_fee", min = 0)

In [46]:
df.shape

(37067652, 22)

In [47]:
df.to_csv('data/yellow_tripdata_combined_cleaned.csv')